In [1]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import jieba

# 一堆句子 (也可以是一篇文章)
documents = []
with open("./cases/bert_finetune/reviews.txt", "r", encoding="utf-8") as file:
    for line in file:
        document = line.split("\t")[0]
        documents.append(document.replace('"', ''))

# 使用 jieba 進行斷詞
docs = [jieba.lcut(doc) for doc in documents]

# 將文本標記為 TaggedDocument 格式
docs = [TaggedDocument(doc, [index]) for index, doc in enumerate(docs)]

'''
Doc2Vec 的架構

dm=1: PV-DM (預設)
dm=0: PV-DBOW

補充：
若你用 dm=0（DBOW）但又希望同時把詞向量也學好，
通常會搭配 dbow_words=1，
因為純 DBOW 訓練主要用「文件向量去預測詞」，
詞向量可能不會被充分更新；
Gensim 的討論也指出 DBOW 的詞向量
在某些情況下可能幾乎維持初始狀態，
若需要詞向量要開 dbow_words=1。
'''

# 建立 Doc2Vec 模型
model = Doc2Vec(
    docs, 
    vector_size=100, 
    window=5, 
    dm=1,
    # dbow_words=1
    min_count=3,
    workers=1,
    epochs=10, # 100/150/200/250 的結果會比較好嗎?
    seed=42
)

d:\miniconda3\envs\nlp\Lib\site-packages\jieba\_compat.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\darren\AppData\Local\Temp\jieba.cache
Loading model cost 1.714 seconds.
Prefix dict has been built successfully.


In [2]:
# 查找索引為 5 的文件向量
vector = model.dv[5]

# 輸出索引為 5 的文件向量
print(vector)

[ 0.0259433   0.03469285 -0.00111666 -0.01868083 -0.0517282   0.05991731
 -0.03608294  0.03787555  0.04327111 -0.04207157 -0.05886843  0.05190066
  0.04035759  0.01424964  0.02846157  0.01896484  0.00540717  0.00413444
  0.01510719 -0.03634094 -0.02237003  0.01434521  0.02002678  0.06969702
  0.1155225   0.00167463 -0.08651707  0.00578223 -0.02078785  0.00402548
 -0.00220096 -0.03807146 -0.00927884 -0.02685548  0.04422115  0.01393797
  0.03549914 -0.03949668  0.06351655  0.03456078  0.02036532  0.06355887
  0.06173351 -0.06935179 -0.00309113  0.04602892  0.03507222 -0.03324396
  0.05610605 -0.0252912  -0.01098299 -0.04104146 -0.02276993 -0.07580889
  0.02499072 -0.04662764 -0.10108487 -0.0250675  -0.0168791  -0.00634301
  0.0999837   0.04270137 -0.01684773  0.04230325 -0.00868521  0.01532395
 -0.01932412 -0.01382952 -0.04301094 -0.03098823 -0.01818291 -0.05266564
  0.00539795 -0.0301584   0.05888811  0.00661432  0.04773217  0.014603
 -0.01838144  0.00171368 -0.02243626  0.03015491 -0.0

In [3]:
# 顯示兩個段落的文字
print(documents[5])
print(docs[5].words)

print(documents[6])
print(docs[6].words)

總的來說，這樣的酒店配這樣的價格還算可以，希望他趕快裝修，給我的客人留些好的印象
['總', '的', '來', '說', '，', '這樣', '的', '酒店', '配', '這樣', '的', '價格', '還算', '可以', '，', '希望', '他', '趕快', '裝修', '，', '給我', '的', '客人', '留些', '好', '的', '印象']
價格比比較不錯的酒店。這次免費升級了，感謝前臺服務員。房子還好，地毯是新的，比上次的好些。早餐的人很多要早去些。
['價格', '比比', '較', '不錯', '的', '酒店', '。', '這次', '免費', '升級', '了', '，', '感謝', '前', '臺', '服務員', '。', '房子', '還好', '，', '地毯', '是', '新', '的', '，', '比', '上次', '的', '好些', '。', '早餐', '的', '人', '很多', '要', '早', '去', '些', '。']


In [4]:
# 儲存模型
model.save('doc2vec.model')

In [5]:
# 讀取模型
model = Doc2Vec.load('doc2vec.model')

In [6]:
# 查找索引為 5 的文件向量
print(documents[5])

# 找前 10 個相似的文件
similar_docs = model.dv.most_similar(5, topn=5)
print(similar_docs)
print()

for doc_index, similarity in similar_docs:
    print("=" * 50)
    print(f"Doc ID: {doc_index}, content: {documents[doc_index]}")
    print(similarity)

總的來說，這樣的酒店配這樣的價格還算可以，希望他趕快裝修，給我的客人留些好的印象
[(2655, 0.7628684639930725), (3866, 0.7176622152328491), (68, 0.713477373123169), (6555, 0.7029337882995605), (6292, 0.7018665671348572)]

Doc ID: 2655, content: 客人很滿意,我們也推薦其他同事入住.價效比好.交通方便,離火車站就5分鐘路程
0.7628684639930725
Doc ID: 3866, content: 房間不錯，朋友很滿意，價格很實惠，服務態度很好
0.7176622152328491
Doc ID: 68, content: 住過好多次這家酒店了，上次來到前臺，服務員能準確的報出我的名字，感覺很親切。四星級就是不一樣。而且當天服務員還給我安排了一間商務單間，房間很新，比我訂的要好價格沒變。說是酒店搞活動，像我們這樣的商務客人都有機會享受，不錯。
0.713477373123169
Doc ID: 6555, content: 不知前面幾位朋友怎麼評價的，反正我自己入住後感覺實在不好，CHACKIN
0.7029337882995605
Doc ID: 6292, content: 大年初一入住的，酒店的裝修不錯，可是前臺服務就一位男生，態度很冷淡，感覺不希望我們入住似的，因為是探親的所以就晚上住一下，所以訂了標準房，他一開口就要我們加錢換好點的房間，我們就是衝那個價去的，如果要出四百多我們選擇其他酒店了，也不會住他家了，地方那麼偏，於是他給我們了普通的房間，那個房間真是小啊，比一百元多的連鎖酒店還小，裝修不錯，床比較舒服，不過下次肯定不會住這家了。不實惠，而且因為在網上看到送迷你吧和可以免費去植物園的，前臺的也沒有交代這些事情，如果沒有在網上看到的人肯定不知道的。
0.7018665671348572


In [7]:
# 查找索引為 5 的文件向量
print(documents[0])

# 查找索引為 843 的文件向量
print(documents[843])

# 計算文檔向量 0 和文檔向量 843 之間的相似度
similarity = model.dv.similarity(5, 843)
print(similarity)

距離川沙公路較近,但是公交指示不對,如果是蔡陸線的話,會非常麻煩.建議用別的路線.房間較為簡單.
價格適中，裝修還可以。早餐尚可。物有所值
0.5960297


# 如何找到合適的 epoch 數量？ (Deprecated)

In [ ]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import jieba, random
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rc('font', family='Microsoft JhengHei')

# 讀取文本
documents = []
with open("./cases/bert_finetune/reviews.txt", "r", encoding="utf-8") as f:
    for line in f:
        doc = line.split("\t")[0].replace('"', '')
        documents.append(doc)

docs = [jieba.lcut(d) for d in documents]
docs = [TaggedDocument(words=ws, tags=[i]) for i, ws in enumerate(docs)]

# 建模（建議明確指定 dm；下面示範 dm=1）
model = Doc2Vec(
    vector_size=100,
    window=5,
    min_count=3,
    dm=1, # 或 dm=0
    workers=4,
    seed=42,
)

# 建立模型的詞彙表與文件標籤索引
model.build_vocab(docs)

# 定義訓練的 epoch 數
num_epochs = 50

# 一開始的學習率
start_alpha = 0.025

# 最後的學習率
end_alpha = 0.0005

# 每跑完一個 epoch，學習率要下降多少（線性下降）
alpha_delta = (start_alpha - end_alpha) / num_epochs

# 用於儲存每個 epoch 的損失值
losses = []

# 檢視每一回合訓練後的損失值
for epoch in range(num_epochs):
    # 每個 epoch 開始前把文件順序打亂
    random.shuffle(docs)

    print("=" * 50)
    print(f'訓練第 {epoch+1} 個 epoch')

    # 線性更新學習率
    a0 = start_alpha - epoch * alpha_delta
    a1 = start_alpha - (epoch + 1) * alpha_delta

    # 訓練
    model.train(
        docs,
        total_examples=model.corpus_count,
        epochs=1,
        start_alpha=a0,
        end_alpha=a1,
        compute_loss=True
    )

    # 取得當前的損失值
    loss = model.get_latest_training_loss()
    losses.append(loss)

    print(f"Epoch {epoch+1:02d} | alpha {a0:.5f}->{a1:.5f} | Loss {loss}")

plt.plot(range(1, num_epochs+1), losses)
plt.xlabel("Epoch")
plt.ylabel("Loss (per-epoch delta)")
plt.title("Doc2Vec 每個 epoch 的 loss")
plt.show()